# Direct Model
## Setup

In [1]:
from data_utils import *
from train import * 

import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

use_cuda = torch.cuda.is_available()
print("GPU Available:", use_cuda)
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

print('using device:', device)

GPU Available: False
using device: cpu


## Load Dataset

In [ ]:
data = load_bolete_data()
print()
for k in data.keys():
    print(k, np.shape(data[k]))

## Split into train and test

In [4]:
X_train, X_test, Y_train, Y_test = get_train_and_test(data, "bolete-labels")

## Define models

In [5]:
def simple_model():
    # copy final model from ass igment 2
    class Flatten(nn.Module):
        def forward(self, x):
            return flatten(x)

    channel_1 = 32
    channel_2 = 24
    channel_3 = 16
    hidden_dim = 150
    torch.manual_seed(0)
    learning_rate = 3e-3 # 1e-2

    model = nn.Sequential(
        nn.Conv2d(C, channel_1, kernel_size=5, padding=2),
        # nn.GroupNorm(4,channel_1),
        nn.BatchNorm2d(channel_1),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1),
        # nn.GroupNorm(4,channel_2),
        nn.BatchNorm2d(channel_2),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Conv2d(channel_2, channel_3, kernel_size=3, padding=1),
        # nn.GroupNorm(4,channel_3),
        nn.BatchNorm2d(channel_3),
        nn.ReLU(),
        nn.Dropout(0.1),
        Flatten(),
        nn.Linear(channel_3 * H * W, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, M)
    )
    optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                        momentum=0.8, nesterov=True)
    return model, optimizer


## Define the loss function

In [6]:
loss_fn = nn.CrossEntropyLoss()

## Cross Validate on training data

In [11]:
model, optimizer = simple_model()

cross_val(X_train,
    Y_train,
    model,
    optimizer,
    loss_fn,
    batch_size=64,
    num_epochs=1,
    show_every=5,
    device=device,
)

torch.Size([1048, 512, 512, 3])
torch.Size([1048])
torch.Size([263, 512, 512, 3])
torch.Size([263])
